# Flask 프레임워크를 이용한 웹 서빙

## Flask 서버 구성
- app.py
- cnn_model.py
- photos-cnn-model.h5
- templates
    - index.html
- static
    - uploads
        - test-salad.jpg

### 1) index.html 

In [ ]:
<!DOCTYPE html>
<head>
    <meta charset="utf8">
    <title>CNN 모델 웹서빙</title>
    <style>
	html { 
		font-size: 20px; 
                         font-weight: bold; 
             }
            img {
                         float: left;
                         width: 40%;
                         height: 50%;
		margin: 5px;
		padding: 5px;
                         border: 2px solid #90C;
            }
            p    {
                         clear: both;
            }
    </style>
<body>
    <div class="container">
    <h1>CNN 모델로 음식 이미지 분류</h1>
    <h2>이미지 업로드 및 CNN 예측</h2>
    <hr>
    {% if filename %}<br>
        <div>
            <img src="{{filename}}" width=500 height=450 align=left>
        </div>   
        <br>       
         <p>예측 : {{label}} &nbsp;&nbsp;  정확도 : {{probability}} &nbsp;&nbsp; 칼로리 : {{cal}}</p>          
     {% endif %}

    <form method="post" action="/upload" enctype="multipart/form-data">
        <input type="file" name="file">
        <input type="submit" value="submit">
    </form>
    </div>
</body>
</head>

### 2) app.py 

- pip install flask

In [ ]:
import cnn_model
from flask import Flask, render_template, request, session, escape, jsonify
import requests
import json, os
from flask import *
from PIL import Image
from tensorflow.keras.models import load_model
import numpy as np

# 테스트 이미지 변형을 위한 변수 선언
im_rows = 32 # 이미지의 높이
im_cols = 32 # 이미지의 너비
im_color = 3 # 이미지의 색공간
in_shape = (im_rows, im_cols, im_color) # 입력 이미지 차원
nb_classes = 3 # 클래스 수

LABELS = ["피자", "스파게티", "스시"] # 레이블의 음식 이름
CALORIES = [266, 157, 228] # 각 레이블별 칼로리

def predict(filename):    
    # 학습된 CNN 모델과 가중치 불러오기
    model = load_model('./model/photos-cnn-model.h5')
    model.load_weights('./model/photos-cnn-weight.hdf5')
    
    # 이미지 읽어 들이기
    print(filename)
    img = Image.open("./"+ filename)
    img = img.convert("RGB") # 색공간 변환하기
    img = img.resize((im_cols, im_rows)) # 크기 변경하기
    
    # 3차원으로 데이터 변환하기
    x = np.asarray(img)
    x = x.reshape(-1, im_rows, im_cols, im_color)
    x = x / 255
    
    # 예측하기
    pre = model.predict([x])[0]
    print(pre)
    idx = pre.argmax()
    per = round(float(pre[idx] * 100),3)
    
    return idx, per

# app 서버 
app = Flask(__name__)

app.config['UPLOAD_FOLDER'] = 'static/uploads'

@app.route('/')
@app.route('/index')
def index():
    return render_template('index.html')

@app.route('/upload', methods = ['POST'])
def upload():
    
    file = request.files['file']
    filename = file.filename
    file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
    img_src = url_for('static', filename = 'uploads/' + filename)

    label, prob = predict(img_src)
    pred = LABELS[label] 
    cal = CALORIES[label] 
    cal = str(cal) + "kcal"
    prob = str(round(prob, 2)) + "%"
    
    return render_template('index.html', filename=img_src, label=pred, probability=prob, cal=cal)

if __name__=='__main__':
    app.run('127.0.0.1', port=5000)    

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [02/Oct/2022 12:53:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2022 12:53:37] "GET /favicon.ico HTTP/1.1" 404 -


/static/uploads/test-pizza.jpg
1/1 [==============================] - 0s 135ms/step


127.0.0.1 - - [02/Oct/2022 12:53:43] "POST /upload HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2022 12:53:43] "GET /static/uploads/test-pizza.jpg HTTP/1.1" 200 -


[9.9709857e-01 2.7578559e-03 1.4362724e-04]


...

127.0.0.1 - - [02/Oct/2022 12:55:25] "GET /static/uploads/test-spaghetti.jpg HTTP/1.1" 200 -


/static/uploads/test-sushi.jpg
1/1 [==============================] - 0s 83ms/step


127.0.0.1 - - [02/Oct/2022 12:55:33] "POST /upload HTTP/1.1" 200 -


[2.1065320e-07 3.7039658e-06 9.9999607e-01]


127.0.0.1 - - [02/Oct/2022 12:55:34] "GET /static/uploads/test-sushi.jpg HTTP/1.1" 200 -
